# 0. Imports and installations

In [1]:
# IMPORTS

import re
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from zipfile import ZipFile
from io import BytesIO
import polars as pl
import multiprocessing
from multiprocessing import Pool, cpu_count
from urllib.parse import urljoin
import concurrent.futures
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

data_dir = 'data'
parquet_dir = os.path.join(data_dir)

# Data preparation

In [2]:
gdelt = pl.read_parquet('data/FILTERED_DATAFRAME_WITH_STATES.parquet')
gdelt = gdelt.drop(["date", "year"])
gdelt = gdelt.rename({"month_year": "period"})
gdelt

state,period,event_code,quad_class,goldstein_scale,avg_tone,actor1_statecode,actor2_statecode,url,title,full_text
list[str],i64,i64,i64,f64,f64,str,str,str,str,str
"[""WA"", ""WI"", … ""MA""]",202105,16,1,-2.0,-8.934073,"""USMO""","""USMO""","""https://www.natlawreview.com/a…","""State of the Law for Business …","""It’s been a year since COVID-1…"
"[""KS"", ""WA"", ""MO""]",202105,141,3,-6.5,-0.808625,"""USMO""","""USMO""","""https://www.kcur.org/health/20…","""Medicaid Expansion Supporters …","""A day after Missouri Gov. Mike…"
"[""NY"", ""MO""]",202105,13,1,0.4,-6.008584,"""USMO""","""USMO""","""https://www.dailystar.co.uk/ne…","""Elderly woman sucker-punched t…","""Elderly woman sucker-punched t…"
null,202002,16,1,-2.0,-8.0,"""USAR""",null,"""https://www.houstonchronicle.c…",null,null
"[""NY"", ""DE"", … ""MA""]",202012,70,2,7.0,0.088106,"""USNH""","""USNH""","""https://www.fosters.com/story/…","""Historically Speaking: Adventu…","""Historically Speaking: Adventu…"
…,…,…,…,…,…,…,…,…,…,…
null,202007,10,1,0.0,-3.512648,"""USAK""",null,"""https://www.wmal.com/news/a-fa…",null,null
null,202012,173,4,-5.0,-6.841617,"""USAZ""","""USUT""","""https://www.startribune.com/ex…",null,null
"[""HI"", ""AK"", … ""AL""]",202012,120,3,-4.0,-3.243243,"""USAK""",null,"""https://www.jewishpress.com/bl…","""Obama: It’s Impossible to Have…","""{Repsoted from the Sultan Knis…"


In [3]:
# Fill missing states with the information from other columns
gdelt = gdelt.with_columns(
    pl.when(pl.col("actor1_statecode").is_null())
      .then(pl.col("actor2_statecode"))
      .otherwise(pl.col("actor1_statecode"))
      .alias("actor1_statecode")
)

gdelt = gdelt.with_columns(
    pl.when(pl.col("actor2_statecode").is_null())
      .then(pl.col("actor1_statecode"))
      .otherwise(pl.col("actor2_statecode"))
      .alias("actor2_statecode")
)

gdelt = gdelt.with_columns([
    pl.col("actor1_statecode").str.replace("^US", "").alias("actor1_statecode"),
    pl.col("actor2_statecode").str.replace("^US", "").alias("actor2_statecode")
])


We check that all the states are valid ones, and drop the ones that aren't.

In [4]:
valid_states = {
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC'
}

gdelt = gdelt.with_columns([
    # Clean 'state' list to keep only valid state codes
    pl.col("state").list.eval(
        pl.element().filter(pl.element().is_in(valid_states))
    ).alias("state"),

    # Replace invalid actor1_statecode with null
    pl.when(pl.col("actor1_statecode").is_in(valid_states))
      .then(pl.col("actor1_statecode"))
      .otherwise(None)
      .alias("actor1_statecode"),

    # Replace invalid actor2_statecode with null
    pl.when(pl.col("actor2_statecode").is_in(valid_states))
      .then(pl.col("actor2_statecode"))
      .otherwise(None)
      .alias("actor2_statecode")
])

We fill in the missing values in "state" with the state names in Actor1 and Actor2 columns

In [5]:
gdelt = gdelt.with_columns(
    pl.when(pl.col("state").is_null())
      .then(
          pl.struct(["actor1_statecode", "actor2_statecode"]).map_elements(
              lambda row: list({row["actor1_statecode"], row["actor2_statecode"]}),
              return_dtype=pl.List(pl.String)
          )
      )
      .otherwise(pl.col("state"))
      .alias("state")
)


Drop rows where all three conditions are true:

- "state" is an empty list

- "actor1_statecode" is null

- "actor2_statecode" is null

If "state" has an the empty list, fill it with Actor1 and/or Actor 2.

In [6]:
gdelt = gdelt.with_columns(
    pl.when(
        (pl.col("state").list.len() == 0) &
        (~pl.col("actor1_statecode").is_null() | ~pl.col("actor2_statecode").is_null())
    )
    .then(
        pl.struct(["actor1_statecode", "actor2_statecode"]).map_elements(
            lambda row: [x for x in {row["actor1_statecode"], row["actor2_statecode"]} if x is not None],
            return_dtype=pl.List(pl.String)
        )
    )
    .otherwise(pl.col("state"))
    .alias("state")
)

gdelt = gdelt.with_columns(
    pl.col("state").list.eval(
        pl.element().filter(pl.element().is_not_null())
    ).alias("state")
)

gdelt = gdelt.filter(
    ~(
        (pl.col("state").list.len() == 0) &
        (pl.col("actor1_statecode").is_null()) &
        (pl.col("actor2_statecode").is_null())
    )
)

gdelt = gdelt.drop(["actor1_statecode", "actor2_statecode", "url"])

We "explode" the "state" column (which contains lists of state codes) so that each row has only one state, while duplicating the rest of the row data.

In [7]:
gdelt = gdelt.explode("state")
gdelt = gdelt.sort(["state", "period"])

In [8]:
gdelt = gdelt.sort(["state", "period"])
gdelt

state,period,event_code,quad_class,goldstein_scale,avg_tone,title,full_text
str,i64,i64,i64,f64,f64,str,str
"""AK""",192001,46,1,7.0,1.149699,"""Alaska Senate president in spo…","""JUNEAU, Alaska - During her fi…"
"""AK""",192001,32,1,4.5,2.31405,"""When the major primaries and c…","""When the major primaries and c…"
"""AK""",192001,42,1,1.9,-3.696789,"""Delta flight attendants say th…","""- Employees of Delta Air Lines…"
"""AK""",192001,42,1,1.9,-1.653944,"""In painting a rosy picture, Go…","""Gov. Mike Dunleavy spoke to th…"
"""AK""",192001,871,2,9.0,-2.219873,"""Who among them is fit to be pr…","""Who among them is fit to be pr…"
…,…,…,…,…,…,…,…
"""WY""",202106,10,1,0.0,-4.634581,"""New Info, Time to Spike Stone-…","""New Info, Time to Spike Stone-…"
"""WY""",202106,120,3,-4.0,-5.128205,"""Liz Cheney blasts the GOP-led …","""- Rep. Liz Cheney of Wyoming c…"
"""WY""",202106,43,1,2.8,0.615385,null,null


We see that some articles have dates like 1920-01 or 2019-01, which is most likely a mistake in labeling the data. We randomly verified several of those and confirmed that they were, indeed, published between 2020-01 and 2021-06. So we replace all instances of '19' for '20' to correct those mistakes.

In [9]:
gdelt = gdelt.with_columns(
    pl.col("period")
    .cast(pl.String)
    .str.replace_all("19", "20")
    .str.replace_all("2010", "2020")
    .str.replace_all("2011", "2020")
    .alias("period")
)

gdelt = gdelt.with_columns(
    pl.col("period").cast(pl.Int64).alias("period")
)

Now we need to collapse the df by "state" and "period". Rules for handling the rest of the columns:
1) event_code: we calculate the proportion of the articles that have event codes that are relevant for terrorism threats:
- 4,	Consult.	Spikes in political consultation sometimes precede crackdowns or unrest
- 5,	Engage in diplomatic cooperation.	Declines in these events may correlate with rising instability
- 7,	Provide military aid (contextual).	Often associated with militarization of conflict regions
- 8,    Provide aid (indirect).	May suggest instability or post-crisis environments
- 9,	Investigate.	Often tied to plots, arrests, or suspicious activities before attacks
- 10,	Demand.	Demands related to grievances or threats
- 12,	Reject, defy, or refuse to cooperate.	Markers of political breakdown or radicalization
- 13	Threaten.	e.g., 130, 131	Verbal threats — often precede political violence or terrorism
- 14	Protest.	e.g., 140–145	Peaceful or violent demonstrations — common precursors to unrest
- 15	Exhibit. force posture	e.g., 150, 151	Mobilization, show of force — suggests preparation for conflict
- 16	Reduce relations.	e.g., 160–166	Diplomatic deterioration, cutting ties — signal of political breakdown
- 17	Coerce.	e.g., 170–177	Includes armed attacks, occupation, forced compliance
- 18	Use unconventional violence (Terrorism).	e.g., 180, 181	Bombings, sabotage, guerrilla warfare — directly overlap with GTD events

We need to collapse the "gdelt" df by "state" and "period". Rules for handling the rest of the columns: 
1) event_code: we calculate the ratio of how many articles in state-period have event codes that are relevant for terrorism threats. we create three new columns with those ratios:
- "cooperation" for codes between 40 and 89
- "investigation" for codes 12 and between 90 and 99
- "instability" for codes between 13 and 18

2) quad_class: we calculate the proportion of the articles that have quad_class of 3 or 4.

3) goldstein_scale: we calculate the proportion of the articles that have goldstein_scale <0.

4) avg_tone:
- we calculate the proportion of the articles that have avg_tone <0.
- we compute the mean of all values per state-month.
- we compute the std of all values per state-month.

5) title: merge all the strings into one
6) full_text: merge all the strings into one

In [10]:
# Standardize event codes to only keep the first two digits
gdelt = gdelt.with_columns(
    pl.when(pl.col("event_code") >= 1000)
      .then(pl.col("event_code") // 100)  # Keep first two digits
      .when(pl.col("event_code") >= 100)
      .then(pl.col("event_code") // 10)   # Keep first two digits
      .otherwise(pl.col("event_code"))
      .alias("event_code")
)

In [11]:
# Define category flags
is_cooperation = (pl.col("event_code") >= 40) & (pl.col("event_code") <= 89)
is_investigation = (pl.col("event_code") == 12) | ((pl.col("event_code") >= 90) & (pl.col("event_code") <= 99))
is_instability = (pl.col("event_code") >= 13) & (pl.col("event_code") <= 18)

# Define value conditions
is_quad_3_or_4 = pl.col("quad_class").is_in([3, 4])
is_goldstein_negative = pl.col("goldstein_scale") < 0
is_avg_tone_negative = pl.col("avg_tone") < 0

# Collapse
gdelt_collapsed = gdelt.group_by(["state", "period"]).agg([
    pl.len().alias("article_count"),

    (is_cooperation.sum().cast(pl.Float64) / pl.len()).alias("cooperation"),
    (is_investigation.sum().cast(pl.Float64) / pl.len()).alias("investigation"),
    (is_instability.sum().cast(pl.Float64) / pl.len()).alias("instability"),

    (is_quad_3_or_4.sum().cast(pl.Float64) / pl.len()).alias("quad_class_3_or_4"),
    (is_goldstein_negative.sum().cast(pl.Float64) / pl.len()).alias("goldstein_negative_ratio"),

    (is_avg_tone_negative.sum().cast(pl.Float64) / pl.len()).alias("avg_tone_negative_ratio"),
    pl.col("avg_tone").mean().alias("avg_tone_mean"),
    pl.col("avg_tone").std().alias("avg_tone_std"),

    pl.col("title").implode().alias("titles_combined"),
    pl.col("full_text").implode().alias("full_text_combined")
])

gdelt_collapsed = gdelt_collapsed.sort(["state", "period"])
del gdelt


We add one more text-based feature - a lexicon of terms related to terrorism (suggested by an LLM based on the relevant academic literature), and we count the term frequency in our state-month corpus.

In [12]:
from collections import Counter

# Set of words to match
target_words = {
    # Violence and conflict
    "attack", "bomb", "explode", "gunfire", "shooting", "massacre", "ambush", "assassinate",
    "kill", "murder", "execute", "detonate", "sniper", "casualty", "hostage", "raid",

    # Terrorism-specific terminology
    "terror", "terrorist", "militant", "extremist", "jihad", "insurgent", "radical", "fundamentalist",
    "al-qaeda", "isis", "islamic state", "boko haram", "hezbollah", "taliban", "caliphate",

    # Target and location indicators
    "embassy", "market", "airport", "station", "checkpoint", "border", "mosque", "church",
    "synagogue", "school", "government", "military", "police", "security", "infrastructure",

    # Planning and operational language
    "plot", "plan", "prepare", "recruit", "train", "smuggle", "fund", "coordinated",
    "sleeper cell", "radicalize", "mission", "deadline", "operation",

    # Ideological and motivational terms
    "infidel", "martyr", "holy war", "fatwa", "blasphemy", "revenge", "oppression",
    "liberation", "sacrifice", "enemy", "zionist", "crusader", "occupation"
}

def count_target_words(text_list):
    filtered_list = [text for text in text_list if text is not None]
    words = " ".join(filtered_list).lower().split()
    word_counts = Counter(words)
    return sum(word_counts[word] for word in target_words if word in word_counts)

# Apply it using .map_elements
gdelt_collapsed = gdelt_collapsed.with_columns(
    pl.col("full_text_combined").map_elements(count_target_words, return_dtype=pl.Int64).alias("target_word_count")
)

gdelt_collapsed = gdelt_collapsed.drop(["titles_combined", "full_text_combined"])

Load a dataset with LDA topic shares. Merge it to the existing Dataframe.

In [13]:
gdelt_lda = pl.read_parquet('data/AGGREGATED_DATAFRAME.parquet')

topic_columns = [col for col in gdelt_lda.columns if col.startswith("topic_")]
columns_to_join = ["state", "period"] + topic_columns
gdelt_lda_subset = gdelt_lda.select(columns_to_join)

gdelt_collapsed = gdelt_collapsed.join(
    gdelt_lda_subset,
    on=["state", "period"],
    how="left"
)

In [16]:
gdelt_collapsed.write_parquet("data/gdelt_features.parquet")
gdelt_collapsed.write_csv("data/gdelt_features.csv")